# Benchmark 01

Les informations sur le benchmark utilisé par le site sont assez vagues:

>Le second benchmark intègre plusieurs étapes de pré-processing et utilise un modèle de Machine Learning optimisé pour prédire le risque de fraude.

>PR-AUC2=0,14

Dans cet exemple de benchmark basique on va s'intéresser juste à la catégorisation des valeurs des champs texte via une régression logistique. Les paramètres pour la régression logistique sont laissés à ceux par défaut de scikit learn.

```py
OneHotEncoder,
LogisticRegression
```

Un peu moins bon que le benchmark proposé, l'aire sous la courbe étant un peu plus faible.

PR-AUC = 0,1292


In [1]:
from utils import *
x_train, y_train, _, x_test = get_data()

# Listes des occurences uniques de valeurs pour les différents types de colonnes
categories = distinct_values_for('category_', x_train)
manufacturers = distinct_values_for('manufacturer_', x_train)
products = distinct_values_for('product_', x_train)
product_codes = distinct_values_for('product_code_', x_train)


In [2]:
# En phase d'évaluation des performances
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                   y_train, test_size=0.2, random_state=42)


# Taux de fraude dans l'ensemble d'entraînement
fraud_rate_in_train =  sum(y_train.fraud)/len(y_train.fraud)*100
# Taux de fraude dans l'ensemble de test
fraud_rate_in_test = sum(y_test.fraud)/len(y_test.fraud)*100

# On s'assure qu'il y a moins de 10% d'écart entre les taux de fraude
assert abs(fraud_rate_in_train-fraud_rate_in_test)/fraud_rate_in_train < 0.1

In [3]:
# Colonnes non textuelles supprimées dans la sélection des colonnes pour le 1_hot_encoding
# id, product_count_i, product_type_count
columns_for_1_hot_encoding = ['category_'+str(i+1) for i in range(24)] \
                             + ['manufacturer_'+str(i+1) for i in range(24)] \
                             + ['product_'+str(i+1) for i in range(24)] \
                             + ['product_code_'+str(i+1) for i in range(24)]

x = x_train[columns_for_1_hot_encoding].to_numpy()

In [4]:
from sklearn.preprocessing import OneHotEncoder
unique_values = [categories for i in range(24)]  \
                + [manufacturers for i in range(24)]  \
                + [products for i in range(24)] \
                + [product_codes for i in range(24)] 
                
encoder = OneHotEncoder(categories=unique_values, handle_unknown='ignore')

encoder.fit(x)

x_dummy = encoder.transform(x)
y = y_train.to_numpy()

In [6]:
# Entrainement

from sklearn.linear_model import LogisticRegression
# y.ravel()
# pour éviter le message 
# DataConversionWarning: A column-vector y was passed when a 1d array was expected.
# Please change the shape of y to (n_samples, ), for example using ravel().
clf = LogisticRegression(solver='liblinear').fit(x_dummy, y.ravel())

# solver="newton-cg"
# un peu mieux en val mais même efficacité en test

In [7]:
# Evaluation 
x_test = x_test[columns_for_1_hot_encoding].to_numpy()
x_test_dummy = encoder.transform(x_test)
y_test = y_test.to_numpy()

y_score = clf.predict_proba(x_test_dummy)[:, 1]


from sklearn.metrics import average_precision_score
average_precision_score(y_test, y_score)

0.1315811859383418

# Proposition de solution

## Entraînement

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from utils import *

x_train, y_train, _, x_test = get_data()

# Listes des occurences uniques de valeurs pour les différents types de colonnes
categories = distinct_values_for('category_', x_train)
manufacturers = distinct_values_for('manufacturer_', x_train)
products = distinct_values_for('product_', x_train)
product_codes = distinct_values_for('product_code_', x_train)

# Colonnes non textuelles supprimées dans la sélection des colonnes pour le 1_hot_encoding
# id, product_count_i, product_type_count
columns_for_1_hot_encoding = ['category_'+str(i+1) for i in range(24)] \
                             + ['manufacturer_'+str(i+1) for i in range(24)] \
                             + ['product_'+str(i+1) for i in range(24)] \
                             + ['product_code_'+str(i+1) for i in range(24)]

x = x_train[columns_for_1_hot_encoding].to_numpy()

unique_values = [categories for i in range(24)]  \
                + [manufacturers for i in range(24)]  \
                + [products for i in range(24)] \
                + [product_codes for i in range(24)] 
                
encoder = OneHotEncoder(categories=unique_values, handle_unknown='ignore')

encoder.fit(x)

x_dummy = encoder.transform(x)
y = y_train.to_numpy()

# Entrainement
clf = LogisticRegression(solver="liblinear").fit(x_dummy, y.ravel())


## Prédictions

In [ ]:
# Evaluation 
x_test_light = x_test[columns_for_1_hot_encoding].to_numpy()
x_test_dummy = encoder.transform(x_test_light)

y_score = clf.predict_proba(x_test_dummy)[:, 1]
# y_score.shape

In [ ]:

create_submission_csv(y_score, x_test.id, 'y_benchmark_01')